In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf

import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark


import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.SparkContext

import org.apache.spark.SparkConf

In [2]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Column

import org.apache.spark.sql.types.{DataType, DateType, TimestampType}

import org.apache.spark.sql.functions._

In [3]:
var spark = SparkSession
.builder()
.appName("Java Spark SQL Homework")
.config("spark.master", "local")
.getOrCreate();
def sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@4eab7bc4
defined function sc

In [4]:
import java.sql.Date
import org.apache.spark.sql.expressions.Window

import java.sql.Date

import org.apache.spark.sql.expressions.Window

In [5]:
var data = spark.read.
option("header", true).
option("inferSchema",true).
csv("data.csv")

data: DataFrame = [device_id: string, country_code: string ... 12 more fields]

In [6]:
var df = data.withColumn("date", data("timestamp").cast(DateType))

df: DataFrame = [device_id: string, country_code: string ... 13 more fields]

Exercise 1

In [7]:
var retention = df.select("date","device_id")//I have selected the columns I need for the calculation

retention: DataFrame = [date: date, device_id: string]

In [8]:
//I am going to join my data with itself, so I have copied the data and renamed its columns.
//I have also added a new column where the number of days is substracted from the "date" column(in our example 1 or 7 days).
val newframe = retention.withColumnRenamed("date","date_after").
withColumnRenamed("device_id","device_id_after").
withColumn("date_back",date_sub(col("date_after"),1))

newframe: DataFrame = [date_after: date, device_id_after: string ... 1 more field]

In [9]:
//Here I have joined the data so that the device_id is the same and the date is equal to my new column with substracted date
var joined = retention.join(newframe,
        retention("device_id") === newframe("device_id_after")
        && retention("date") === newframe("date_back"),"left")

joined: DataFrame = [date: date, device_id: string ... 3 more fields]

In [10]:
//this shows the number of active users each day
var active = (joined.groupBy("date").agg(countDistinct("device_id").as("active")).orderBy("date"))

active: Dataset[Row] = [date: date, active: bigint]

In [11]:
//this shows the number of returned users
var returned = (joined.groupBy("date").agg(countDistinct("device_id_after").as("returned")).orderBy("date"))

returned: Dataset[Row] = [date: date, returned: bigint]

In [12]:
//joining those two DFs by "date" and calculating the percent, we get the retention table for each day
var ret = active.join (returned,Seq("date"),"left").orderBy("date").
withColumn("percent",round(col("returned")/col("active")*100,2))

ret: DataFrame = [date: date, active: bigint ... 2 more fields]

In [13]:
ret.show

+----------+------+--------+-------+
|      date|active|returned|percent|
+----------+------+--------+-------+
|2018-02-02|    98|      35|  35.71|
|2018-02-03|    35|      21|   60.0|
|2018-02-04|    39|      24|  61.54|
|2018-02-05|    43|      24|  55.81|
|2018-02-06|    35|      23|  65.71|
|2018-02-07|    34|      20|  58.82|
|2018-02-08|    37|      15|  40.54|
|2018-02-09|    28|      15|  53.57|
|2018-02-10|    32|      17|  53.13|
|2018-02-11|    26|      19|  73.08|
|2018-02-12|    34|      23|  67.65|
|2018-02-13|    29|      22|  75.86|
|2018-02-14|    46|      19|   41.3|
|2018-02-15|    28|      15|  53.57|
|2018-02-16|    31|      12|  38.71|
|2018-02-17|    19|       0|    0.0|
+----------+------+--------+-------+



In [14]:
//To calculate the average retention rate we should drop the last 1 day, because that's the last day of our frame, and the 
//retention rate is 0.(when calculating the D7 retention, we should drop last 7 days)
var av_retention = ret.filter(col("date") =!= "2018-02-17").agg(avg("percent").as("retention")).show

+------------------+
|         retention|
+------------------+
|55.666666666666664|
+------------------+



In [15]:
//To calculate the D7 retention we can repeat the same only changing the number of days substracted from the date
val newframe = retention.withColumnRenamed("date","date_after").
withColumnRenamed("device_id","device_id_after").
withColumn("date_back",date_sub(col("date_after"),7))

newframe: DataFrame = [date_after: date, device_id_after: string ... 1 more field]

In [16]:
var joined = retention.join(newframe,
        retention("device_id") === newframe("device_id_after")
        && retention("date") === newframe("date_back"),"left")

joined: DataFrame = [date: date, device_id: string ... 3 more fields]

In [17]:
var active = (joined.groupBy("date").agg(countDistinct("device_id").as("active")).orderBy("date"))

active: Dataset[Row] = [date: date, active: bigint]

In [18]:
var returned = (joined.groupBy("date").agg(countDistinct("device_id_after").as("returned")).orderBy("date"))

returned: Dataset[Row] = [date: date, returned: bigint]

In [19]:
var ret = active.join (returned,Seq("date"),"left").orderBy("date").
withColumn("percent",round(col("returned")/col("active")*100,2))

ret: DataFrame = [date: date, active: bigint ... 2 more fields]

In [20]:
ret.show

+----------+------+--------+-------+
|      date|active|returned|percent|
+----------+------+--------+-------+
|2018-02-02|    98|      28|  28.57|
|2018-02-03|    35|      13|  37.14|
|2018-02-04|    39|      18|  46.15|
|2018-02-05|    43|      23|  53.49|
|2018-02-06|    35|      12|  34.29|
|2018-02-07|    34|      20|  58.82|
|2018-02-08|    37|      14|  37.84|
|2018-02-09|    28|      14|   50.0|
|2018-02-10|    32|       7|  21.88|
|2018-02-11|    26|       0|    0.0|
|2018-02-12|    34|       0|    0.0|
|2018-02-13|    29|       0|    0.0|
|2018-02-14|    46|       0|    0.0|
|2018-02-15|    28|       0|    0.0|
|2018-02-16|    31|       0|    0.0|
|2018-02-17|    19|       0|    0.0|
+----------+------+--------+-------+



In [21]:
var av_retention = ret.filter(col("date") < "2018-02-11").agg(avg("percent")).show

+------------------+
|      avg(percent)|
+------------------+
|40.908888888888896|
+------------------+



In [22]:
//The automation of this method can look like this.
//The function takes data and the number of days retention should be calculated for
def retention(data:DataFrame,days:Int):Unit ={
  var df = data.withColumn("date", data("timestamp").cast(DateType))
  var retention = df.select("date","device_id")
  val newframe = retention.withColumnRenamed("date","date_after").
  withColumnRenamed("device_id","device_id_after").
  withColumn("date_back",date_sub(col("date_after"),days))
  var joined = retention.join(newframe,
        retention("device_id") === newframe("device_id_after")
        && retention("date") === newframe("date_back"),"left")
  var ret = (joined.groupBy("date").agg(countDistinct("device_id").as("active")).orderBy("date")).
  join ((joined.groupBy("date").agg(countDistinct("device_id_after").as("returned")).orderBy("date")),Seq("date"),"left").
  orderBy("date").
  withColumn("percent",round(col("returned")/col("active")*100,2))
  val window = Window.orderBy(df("date").desc)
  var retent = ret.withColumn("rank", rank().over(window)).
  filter(col("rank") > days). 
  drop("rank").agg(round(avg("percent"),2).as("retention")).show
}

defined function retention

In [23]:
retention(data,1)

+---------+
|retention|
+---------+
|    55.67|
+---------+



In [24]:
retention(data,7)

+---------+
|retention|
+---------+
|    40.91|
+---------+



exercise 2

In [82]:
//here can be used the function for the previos example, only adding the column we want as an argument

In [25]:
def retentionEvent(data:DataFrame,days:Int,column:String):Unit ={
  var df = data.withColumn("date", data("timestamp").cast(DateType))
  var retentt = df.select("date","device_id","event_type")
  var retention = retentt.filter(col("event_type") === column)
  var newframe = retention.withColumnRenamed("date","date_after").
  withColumnRenamed("device_id","device_id_after").
  withColumnRenamed("event_type","event").
  withColumn("date_back",date_sub(col("date_after"),days))
  var joined = retention.join(newframe,
        retention("device_id") === newframe("device_id_after")
        && retention("date") === newframe("date_back"),"left")
  var ret = (joined.groupBy("date").agg(countDistinct("device_id").as("active")).orderBy("date")).
  join ((joined.groupBy("date").agg(countDistinct("device_id_after").as("returned")).orderBy("date")),Seq("date"),"left").
  orderBy("date").
  withColumn("percent",round(col("returned")/col("active")*100,2))
  val window = Window.orderBy(df("date").desc)
  var retent = ret.withColumn("rank", rank().over(window)).
  filter(col("rank") > days). 
  drop("rank").agg(round(avg("percent"),2).as("retention")).show
}

defined function retentionEvent

In [26]:
retentionEvent(data,1,"edit_text_apply")

+---------+
|retention|
+---------+
|    24.64|
+---------+



In [27]:
retentionEvent(data,1,"effect_apply")

+---------+
|retention|
+---------+
|    35.13|
+---------+



In [28]:
retentionEvent(data,1,"edit_item_open")

+---------+
|retention|
+---------+
|    41.59|
+---------+



In [29]:
retentionEvent(data,1,"edit_sticker_apply")

+---------+
|retention|
+---------+
|     7.69|
+---------+



Exercise 3

In [30]:
var like = df.filter(col("event_type") === "photo_like")

like: Dataset[Row] = [device_id: string, country_code: string ... 13 more fields]

In [31]:
like.groupBy("date").agg(countDistinct("device_id")).orderBy("date").show
//as our data is not large, there are only two users who did photo_like

+----------+-------------------------+
|      date|count(DISTINCT device_id)|
+----------+-------------------------+
|2018-02-03|                        1|
|2018-02-08|                        1|
+----------+-------------------------+



Exercise 4

In [32]:
var next = df.withColumn("next_event",lead("event_type",1)over Window.partitionBy("device_id").orderBy("timestamp")).
filter(col("event_type") === "effect_try")

next: Dataset[Row] = [device_id: string, country_code: string ... 14 more fields]

In [33]:
var ef_ap = next.filter(col("next_event") === "effect_apply")

ef_ap: Dataset[Row] = [device_id: string, country_code: string ... 14 more fields]

In [34]:
var rate = ef_ap.agg(countDistinct("device_id")).take(1)(0)(0).toString.toDouble/
            next.agg(countDistinct("device_id")).take(1)(0)(0).toString.toInt*100

rate: Double = 71.42857142857143

In [35]:
//automation
def conversion(data:DataFrame,column1:String,column2:String):Unit ={
  var df = data.withColumn("date", data("timestamp").cast(DateType))
  var next = df.withColumn("next_event",lead("event_type",1)over Window.partitionBy("device_id").orderBy("timestamp")).
  filter(col("event_type") === column1)
  var ef_ap = next.filter(col("next_event") === column2)
  println(ef_ap.agg(countDistinct("device_id")).take(1)(0)(0).toString.toDouble/
            next.agg(countDistinct("device_id")).take(1)(0)(0).toString.toInt*100)
}

defined function conversion

In [36]:
conversion(data,"effect_try","effect_apply")

71.42857142857143


Exercise 5

In [37]:
def eff_app = df.filter(col("event_type") === "effect_apply")

defined function eff_app

In [38]:
var ef = eff_app.groupBy("date").agg(countDistinct("device_id").as ("users")).orderBy(col("users").desc)

ef: Dataset[Row] = [date: date, users: bigint]

In [39]:
ef.show

+----------+-----+
|      date|users|
+----------+-----+
|2018-02-02|   21|
|2018-02-12|   13|
|2018-02-04|   12|
|2018-02-11|   10|
|2018-02-15|   10|
|2018-02-03|    9|
|2018-02-10|    9|
|2018-02-05|    8|
|2018-02-08|    8|
|2018-02-14|    8|
|2018-02-06|    8|
|2018-02-13|    7|
|2018-02-09|    6|
|2018-02-17|    5|
|2018-02-16|    5|
|2018-02-07|    5|
+----------+-----+



In [40]:
ef.describe().show

+-------+-----------------+
|summary|            users|
+-------+-----------------+
|  count|               16|
|   mean|              9.0|
| stddev|3.966526608171604|
|    min|                5|
|    max|               21|
+-------+-----------------+



In [41]:
ef.withColumn("percentile", percent_rank() over Window.orderBy("users")).
    filter(col("percentile")>= 0.5).
    limit(1).show
//we can see that 0.5 percentile which is the median is equal to the mean (we can assume that we don't have outliers)

+----------+-----+----------+
|      date|users|percentile|
+----------+-----+----------+
|2018-02-10|    9|       0.6|
+----------+-----+----------+



In [44]:
//to find out whether we have outliers, I will use the outlier formula. 
//for that I should calculate the Q1,Q3,Interquantile range, and later see if my data has outliers
var q1 = ef.withColumn("percentile", percent_rank() over Window.orderBy("users")).
    filter(col("percentile")>= 0.25).
    limit(1).take(1)(0)(1).toString.toInt
var q3 = ef.withColumn("percentile", percent_rank() over Window.orderBy("users")).
    filter(col("percentile")>= 0.75).
    limit(1).take(1)(0)(1).toString.toInt
var interquantile = q3 - q1

q1: Int = 7
q3: Int = 12
interquantile: Int = 5

In [45]:
var lower_outlier = q1 - (3*interquantile)// as we don't have a number smaller than -8, we have no lower ouliers

lower_outlier: Int = -8

In [46]:
var upper_outlier =  q3 + (3*interquantile)//as we don't have a number greater than 27, we have no upper outliers as well

upper_outlier: Int = 27

Exercise 6

In [47]:
var window = Window.partitionBy("device_id").orderBy("timestamp")
df.
withColumn("event_2", lead("event_type", 1) over window).
withColumn("event_3", lead("event_type", 2) over window).
withColumn("event_4", lead("event_type", 3) over window).
withColumn("event_5", lead("event_type", 4) over window).
withColumn("events",concat(col("event_type"),lit(","),col("event_2"),lit(","),col("event_3"),lit(","),col("event_4"),lit(","),col("event_5"))).
select("device_id","events").
groupBy("events").
agg(countDistinct("device_id").as("users")).
orderBy(col("users").desc).na.drop.
withColumn("percent",round(col("users") / df.agg(countDistinct("device_id")).take(1)(0)(0).toString.toInt*100,2))
.show(5,false)

+-----------------------------------------------------------------------------------------------+-----+-------+
|events                                                                                         |users|percent|
+-----------------------------------------------------------------------------------------------+-----+-------+
|effect_try,effect_try,effect_try,effect_try,effect_try                                         |42   |42.86  |
|native_ad_response,native_ad_waterfall,native_ad_request,native_ad_response,native_ad_waterfall|41   |41.84  |
|shop_response,shop_request,shop_response,shop_request,shop_response                            |38   |38.78  |
|native_ad_response,native_ad_waterfall,native_ad_response,native_ad_waterfall,native_ad_request|37   |37.76  |
|shop_request,shop_response,shop_request,shop_response,shop_request                             |35   |35.71  |
+-----------------------------------------------------------------------------------------------+-----+-

window: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2491e84

In [197]:
/*To describe one session of the user I think we should pay attantion:
1.to the number of events done during the session,
2.to the frequencies of each event, so we can calculate the event user does the most or the least,
3.to the firts events done, which I think indicates what is the main reason user opened the app,
4.if after trying some tool user applied it (e.g. effect_try,effect_apply)
5.if after editing the photo user saved it in the phone,uploaded it or just closed
the app, without saving the changes done.*/